# Install Required Software Environment

## Step 1: Create a `work_dir` context manager

This is useful when you want to run a command in a working directory and then automatically change back to your original directory 

In [1]:
import os
from contextlib import contextmanager
from pathlib import Path

@contextmanager
def work_dir(new_dir):
    old_dir = os.getcwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(old_dir)

## Step 2: Install the `isce3` environment with Pixi

In [2]:
with work_dir(Path.cwd().parent):
    !pixi install -e isce3

⠁ preparing packages   [╾───────────────────]  4/362 libgdal-core (+33)    
⠁ preparing packages   [━━──────────────────] 35/362 pandas (+153)  
⠉ preparing packages   [━━━─────────────────] 140/362 scipy (+123)  
⠉ preparing packages   [━━━━━━━╾────────────] 244/362 python (+94)  
⠙ preparing packages   [━━━━━━━━━━━━━━──────] 341/362 python (+20)  
⠙ preparing packages   [━━━━━━━━━━━━━━━━━╾──] 359/362 nodejs (+2)   
⠚ preparing packages   [━━━━━━━━━━━━━━━━━╾──] 359/362 nodejs (+2)   
⠚ preparing packages   [━━━━━━━━━━━━━━━━━╾──] 359/362 nodejs (+2)   
⠒ preparing packages   [━━━━━━━━━━━━━━━━━╾──] 359/362 nodejs (+2)   
⠒ preparing packages   [━━━━━━━━━━━━━━━━━━╾─] 360/362 nodejs (+1)   
⠂ preparing packages   [━━━━━━━━━━━━━━━━━━╾─] 360/362 nodejs (+1)   
⠂ preparing packages   [━━━━━━━━━━━━━━━━━━╾─] 360/362 nodejs (+1)   
⠂ preparing packages   [━━━━━━━━━━━━━━━━━━╾─] 360/362 nodejs (+1) ng
⠂ preparing packages   [━━━━━━━━━━━━━━━━━━╾─] 360/362 nodejs (+1)   
⠒ preparing packages   [━━━

## Step 3: Register the `isce3` environment's Python kernel with `ipykernel`

In [3]:
env_name = "isce3"
display_name = f'"{env_name} (Python)"'

!pixi run -e isce3 python -m ipykernel install \
  --user \
  --name $env_name \
  --display-name $display_name

⠁ activating environment                                                        Installed kernelspec isce3 in /home/jovyan/.local/share/jupyter/kernels/isce3


## Step 4: Wrap the environment's `kernel.json`'s `ipykernel_launcher` call in a `pixi run`

This ensures that shell commands executed from inside a notebook with `!` run in the notebook kernel’s environment. This works by launching the entire Jupyter kernel process inside the Pixi environment, so the kernel’s PATH, environment variables, and Python executable all come from that Pixi environment, not from the parent JupyterLab environment.

In [4]:
from jupyter_client.kernelspec import KernelSpecManager
import json

ksm = KernelSpecManager()
spec = ksm.get_kernel_spec(env_name)
kernel_dir = Path(spec.resource_dir)
kernel_json = kernel_dir / "kernel.json"

data = json.loads(kernel_json.read_text())
orig_argv = data.get("argv", [])

new_argv = [
    "pixi",
    "run",
    "--manifest-path",
    str(Path.cwd().parent),
    "-e",
    env_name,
] + orig_argv

data["argv"] = new_argv
kernel_json.write_text(json.dumps(data, indent=2))
print(f"Updated kernel.json at {kernel_json} with Pixi wrapper.")
print("argv:", data["argv"])

Updated kernel.json at /home/jovyan/.local/share/jupyter/kernels/isce3/kernel.json with Pixi wrapper.
argv: ['pixi', 'run', '--manifest-path', '/home/jovyan/NISAR_GCOV_Cookbook', '-e', 'isce3', '/home/jovyan/NISAR_GCOV_Cookbook/.pixi/envs/isce3/bin/python', '-Xfrozen_modules=off', '-m', 'ipykernel_launcher', '-f', '{connection_file}']


## Step 5: (Optional) Delete the environment and remove its `kernelspec`

In [5]:
with work_dir(Path.cwd().parent):
    !pixi clean
    !jupyter kernelspec remove isce3 -y

  removed /home/jovyan/NISAR_GCOV_Cookbook/.pixi/envs                           Removed /home/jovyan/.local/share/jupyter/kernels/isce3
